In [15]:
import glob
import os
import cv2
import util
import numpy as np
import matplotlib
from tqdm import tqdm
from PIL import Image
import shutil
import re
import util.util_classifier_pediatric as util_classifier_pediatric
import util.util as util
import util.util_preprocessing as util_preprocessing
import util.util_testing as util_testing
from tqdm import tqdm
import util.config as config

ModuleNotFoundError: No module named 'torch'

In [12]:
dir(util)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [3]:
!pip install ~/tqdm-4.66.1-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
Processing /home/brandon.nelson/tqdm-4.66.1-py3-none-any.whl


In [ ]:
DENSITY = "fatty"
SIZE = "7.0"
LESIONDENSITY = "1.06"
DETECTOR = "SIM"
DOSE = "2.22e10"
l_detectorTypes = [DETECTOR]

sourceDir00,sourceDir0 = util_preprocessing.get_source_dirs(config.dir_training_data,LESIONDENSITY,DENSITY,SIZE,DETECTOR,DOSE)
saveDir00 = util_preprocessing.get_save_dir(config.dir_training_data_preprocessed, DENSITY,SIZE,LESIONDENSITY,DOSE,DETECTOR)
nickname = util_preprocessing.get_model_nickname(DENSITY,SIZE,DETECTOR,LESIONDENSITY,DOSE)
saveDir00

In [ ]:
preprocess_images = True
split_train_test = True
train_classifier = True
create_evaluations_dicts = True

In [ ]:
#Copy the images and create the "annotation.txt"
if preprocess_images:
    saveDir = saveDir00
    saveDir_images = saveDir + "/" + "images/"
    os.makedirs(saveDir_images, exist_ok=True)
    annotationsFile = saveDir + "/annotations.txt"

    annotations_list = []
    for detectorType in l_detectorTypes:
        examples = glob.glob(sourceDir0 + "/*/*/projection_DM*.raw")
        for example in tqdm(examples):
            saveName = (
                example.replace(sourceDir00, "")
                .replace("/", "_")
                .replace(".raw", "")
            )
            file_outputProjection = (
                os.path.dirname(example) + "/" + "output_projection.out"
            )
            stringToSearch = "adipose"
            all_results = util.searchForLine(file_outputProjection, stringToSearch)
            dose = float(all_results[1].split("\t")[2])
            # print('DOSE ' + str(dose))
            filename_mhd = example.replace(".raw", ".mhd")

            lesion_present = util_preprocessing.get_lesion_label(filename_mhd)
            if lesion_present > 0:
                label = "lesion"
            else:
                label = "nolesion"

            # read image
            savePath = example.replace(sourceDir00, saveDir00).replace(
                ".raw", ".png"
            )
            tmp = util_preprocessing.preprocess_raw_file(filename_mhd)
            np.save(saveDir_images + "/" + saveName + ".npy", tmp)

            annotation = (
                saveName
                + ".npy"
                + " "
                + str(DENSITY)
                + " "
                + str(detectorType)
                + " "
                + str(dose)
                + " "
                + label
            )
            annotations_list.append(annotation)

    with open(annotationsFile, "w") as f:
        for line in annotations_list:
            f.write(f"{line}\n")

In [ ]:
if split_train_test:
    dirPath = saveDir00
    saveDir_trainValTest = dirPath + "_basic_train_val_test/"

    flImages = open(dirPath + "annotations.txt", "r")
    lines = flImages.readlines()
    flImages.close()

    
    lines = [line.strip() for line in lines]
    print(lines[0])

    lines_notP = [lines[i] for i in range(len(lines)) if i < int(len(lines)*0.7)] # %70 training
    print(len(lines_notP))

    lines_P = [lines[i] for i in range(len(lines)) if (i >= int(len(lines)*0.7) and i < int(len(lines)*0.85))] # %15 validation
    print(len(lines_P))


    lines_P_t = [lines[i] for i in range(len(lines)) if i >= int(len(lines)*0.85)] # %15 test
    print(len(lines_P_t))


    x_train = lines_notP
    x_val = lines_P
    x_test = lines_P_t

    os.makedirs(saveDir_trainValTest, exist_ok=True)
    os.makedirs(saveDir_trainValTest + "/train/withlesion/", exist_ok=True)
    os.makedirs(saveDir_trainValTest + "/train/nolesion/", exist_ok=True)
    os.makedirs(saveDir_trainValTest + "/val/withlesion/", exist_ok=True)
    os.makedirs(saveDir_trainValTest + "/val/nolesion/", exist_ok=True)
    os.makedirs(saveDir_trainValTest + "/test/withlesion/", exist_ok=True)
    os.makedirs(saveDir_trainValTest + "/test/nolesion/", exist_ok=True)

    for i in tqdm(range(len(x_train))):
        line = x_train[i]
        imgname = line.split(" ")[0]
        if not os.path.isfile(dirPath + "images/" + imgname):
            print("PROBLEM")
        if "nolesion" in line:
            shutil.copyfile(
                dirPath + "images/" + imgname,
                saveDir_trainValTest + "/train/nolesion/" + imgname,
            )
        else:
            shutil.copyfile(
                dirPath + "images/" + imgname,
                saveDir_trainValTest + "/train/withlesion/" + imgname,
            )

    for i in tqdm(range(len(x_val))):
        line = x_val[i]
        imgname = line.split(" ")[0]
        if not os.path.isfile(dirPath + "images/" + imgname):
            print("PROBLEM")
        if "nolesion" in line:
            shutil.copyfile(
                dirPath + "images/" + imgname,
                saveDir_trainValTest + "/val/nolesion/" + imgname,
            )
        else:
            shutil.copyfile(
                dirPath + "images/" + imgname,
                saveDir_trainValTest + "/val/withlesion/" + imgname,
            )

    for i in tqdm(range(len(x_test))):
        line = x_test[i]
        imgname = line.split(" ")[0]
        if not os.path.isfile(dirPath + "images/" + imgname):
            print("PROBLEM")
        if "nolesion" in line:
            shutil.copyfile(
                dirPath + "images/" + imgname,
                saveDir_trainValTest + "/test/nolesion/" + imgname,
            )
        else:
            shutil.copyfile(
                dirPath + "images/" + imgname,
                saveDir_trainValTest + "/test/withlesion/" + imgname,
            )

print("Number of training images, no lesion: ", len(os.listdir(saveDir_trainValTest + "train/nolesion/")))
print("Number of training images, with lesion: ", len(os.listdir(saveDir_trainValTest + "train/withlesion/")))
print("Number of validation images, no lesion: ", len(os.listdir(saveDir_trainValTest + "val/nolesion/")))
print("Number of validation images, with lesion: ", len(os.listdir(saveDir_trainValTest + "val/withlesion/")))
print("Number of test images, no lesion: ", len(os.listdir(saveDir_trainValTest + "test/nolesion/")))
print("Number of test images, with lesion: ", len(os.listdir(saveDir_trainValTest + "test/withlesion/")))

In [ ]:
#Train model
if train_classifier:
    util_classifier_pediatric.train_model(saveDir_trainValTest, nickname, nreps=10)

In [ ]:
#load models
saveDir00 = saveDir_trainValTest
if config.FLOAT:
    model_run_file = saveDir00 +'/float/'+'train.log'
else:
    model_run_file = saveDir00 +'/uint8/'+'train.log'
model_run_file
model_runs = [model_run_file]
model_runs

In [ ]:
#Get the example ID  in test data
test_log_list = []
test_log_ID_list = []
testDataDir_nolesion = saveDir_trainValTest + '/test/nolesion'
print(testDataDir_nolesion)
testDataDir_withlesion = saveDir_trainValTest +'/test/withlesion'
for log_name in os.listdir(testDataDir_nolesion):
    test_log_list.append(log_name)
for log_name in os.listdir(testDataDir_withlesion):
    test_log_list.append(log_name)
for item in test_log_list:
    test_log_ID_list.append(item.split("projection_DM")[1].split('.npy')[0])
    
print("Number of test images:", len(test_log_list))

In [ ]:
if create_evaluations_dicts:
    dict_dir = dirPath + '/dicts/'
    print(dict_dir)
    os.makedirs(dict_dir, exist_ok=True)
    util_testing.run_dict_script(test_log_list[0:1], test_log_ID_list, model_runs, dict_dir,logDir='log_victre/',NEXAMPLES=len(test_log_ID_list)) #log_testing